# PrecisionVoice - Google Colab Runner

This notebook allows you to run the [PrecisionVoice](https://github.com/thichuong/PrecisionVoice) application directly in Google Colab.

### Instructions
1.  **Runtime Change**: Go to `Runtime` -> `Change runtime type` and make sure **T4 GPU** (or better) is selected.
2.  **Run All**: You can select `Runtime` -> `Run all` or run each cell step-by-step.
3.  **Public URL**: Look for the `ngrok` public URL in the final cell output to access the web interface.

In [ ]:
# @title 1. Check GPU Availability
import torch

if torch.cuda.is_available():
    print(f"GPU Detected: {torch.cuda.get_device_name(0)}")
else:
    print("WARNING: No GPU detected. This application requires a GPU to run efficiently.")
    print("Please go to Runtime -> Change runtime type -> Hardware accelerator -> T4 GPU")

GPU Detected: Tesla T4


In [ ]:
# @title 2. Clone Repository
import os

# Clean up previous run if exists
if os.path.exists("PrecisionVoice"):
    %cd /content
    !rm -rf PrecisionVoice

!git clone https://github.com/thichuong/PrecisionVoice.git
%cd PrecisionVoice
print("Repository cloned successfully.")

Cloning into 'PrecisionVoice'...
remote: Enumerating objects: 94, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 94 (delta 34), reused 88 (delta 28), pack-reused 0 (from 0)
Receiving objects: 100% (94/94), 35.72 KiB | 5.10 MiB/s, done.
Resolving deltas: 100% (34/34), done.
/content/PrecisionVoice/PrecisionVoice/PrecisionVoice
Repository cloned successfully.


In [ ]:
# @title 3. Install System Dependencies
import subprocess

# Installing dependencies defined in Dockerfile (ffmpeg, libsndfile)
print("Installing system dependencies... (This may take a moment)")

# Update and install (suppressing harmless R-repo warnings common in Colab)
!apt-get update -y > /dev/null 2>&1
!apt-get install -y ffmpeg libsndfile1 > /dev/null 2>&1

# Verify installation
try:
    subprocess.run(["ffmpeg", "-version"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)
    print("✅ System dependencies (ffmpeg, libsndfile1) installed successfully.")
except Exception as e:
    print("❌ Warning: Potential installation issue. If the app fails, try running '!apt-get install -y ffmpeg' manually.")

Installing system dependencies... (This may take a moment)
✅ System dependencies (ffmpeg, libsndfile1) installed successfully.


In [ ]:
# @title 4. Install Python Dependencies
# Force upgrade torch, torchvision, torchaudio to ensure compatibility
!pip install -U torch torchvision torchaudio

!pip install -r requirements.txt
# Install pyngrok to expose the local server to the internet
!pip install pyngrok
print("Python dependencies installed.")

In [ ]:
# @title 5. Setup Environment (.env)
# Creating a default .env file. You can modify this cell to add your specific keys.

env_content = """
PORT=7860
LOG_LEVEL=INFO

# Audio Processing
NOISE_REDUCTION_LEVEL=5.0
VAD_THRESHOLD=0.5
VAD_MIN_SPEECH_DURATION_MS=250
VAD_MIN_SILENCE_DURATION_MS=500
MERGE_THRESHOLD_S=1.5
"""

with open(".env", "w") as f:
    f.write(env_content)

print(".env file created with default settings.")

.env file created with default settings.


In [ ]:
# @title 6. Run Application
import threading
import time
import os
from google.colab.output import serve_kernel_port_as_iframe, serve_kernel_port_as_window
from pyngrok import ngrok

# FORCE KILL any existing ngrok processes to free up the auth token session
print("Cleaning up previous sessions...")
!killall ngrok 2>/dev/null 
ngrok.kill()

# Set your authtoken (Ensure this matches the one in your Ngrok dashboard)
ngrok.set_auth_token("NGROK_TOKEN")

port = 8000

def start_ngrok():
    # Wait a bit for the server to start
    time.sleep(5)
    try:
        # Connect to the port
        public_url = ngrok.connect(port).public_url
        print(f"\n🚀 Ngrok Public URL: {public_url}\n")
    except Exception as e:
        print(f"Ngrok error: {e}")

# Start ngrok in a background thread
threading.Thread(target=start_ngrok, daemon=True).start()

# Serve the application directly in the notebook cell
serve_kernel_port_as_iframe(port, height=900)

# Also provide a link to open in a new tab via proxy
serve_kernel_port_as_window(port, path="/")

# Run the Uvicorn server
print(f"Starting server on port {port}...")
!uvicorn app.main:app --host 0.0.0.0 --port {port}

Cleaning up previous sessions...


<IPython.core.display.Javascript object>

Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

Starting server on port 8000...
INFO:     Started server process [25608]
INFO:     Waiting for application startup.
2026-01-04 03:10:59,288 - app.main - INFO - Starting PrecisionVoice application...
2026-01-04 03:10:59,292 - app.main - INFO - Device: cuda
2026-01-04 03:10:59,292 - app.main - INFO - Whisper model: kiendt/PhoWhisper-large-ct2
2026-01-04 03:10:59,293 - app.main - INFO - Diarization model: pyannote/speaker-diarization-3.1
2026-01-04 03:10:59,293 - app.main - INFO - Preloading Whisper model...
2026-01-04 03:10:59,293 - app.services.transcription - INFO - Loading Whisper model: kiendt/PhoWhisper-large-ct2
2026-01-04 03:10:59,293 - app.services.transcription - INFO - Device: cuda, Compute type: float16

🚀 Ngrok Public URL: https://tandy-pileous-biologically.ngrok-free.dev

2026-01-04 03:11:02,736 - app.services.transcription - INFO - Whisper model loaded successfully
2026-01-04 03:11:02,737 - app.main - WARNING - HF_TOKEN not set, diarization will not be available
2026-01-04 